In [ ]:
from openai import OpenAI
import base64
from openai import OpenAI
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO

client = OpenAI(api_key='...')  

models = client.models.list()

print("Available Models:")
for model in models.data:
    print(model.id)

In [ ]:
import os
import json
from openai import OpenAI
from PIL import Image
import base64

# 配置
adv_image_dir = '...'
vqa_json_path = 'coco300_vqa_main.json'
output_json_path = 'gpt4o_eval_res.json'
client = OpenAI(api_key='...')



def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def get_gpt4o_response(image_path, questions):

    base64_image = encode_image(image_path)
    
    responses = []

    prompt = "Describe the image briefly in one sentence."
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        },
                    },
                ],
            }
        ],
        max_tokens=200,
    )
    responses.append(response.choices[0].message.content)

    for question in questions:
        question = "In this image, " + question
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": question},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            },
                        },
                    ],
                }
            ],
            max_tokens=200,
        )
        responses.append(response.choices[0].message.content)
    
    return responses

def main():

    with open(vqa_json_path, 'r') as f:
        vqa_data = json.load(f)

    image_to_questions = {item['image']: item['vqa'] for item in vqa_data}
    
    results = []

    for filename in os.listdir(adv_image_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(adv_image_dir, filename)
            
            questions = image_to_questions.get(filename, ["", "", ""])
            
            print(f"Processing {filename}...")
            try:
                responses = get_gpt4o_response(image_path, questions)
                
                result = {
                    "filename": filename,
                    "adversarial_response_1": responses[0],
                    "adversarial_response_2": responses[1],
                    "adversarial_response_3": responses[2],
                    "adversarial_response_4": responses[3]
                }
                results.append(result)
                
                with open(output_json_path, 'w') as f:
                    json.dump(results, f, indent=2)
                    
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")
                continue
    
    print(f"Processing completed. Results saved to {output_json_path}")

if __name__ == "__main__":
    main()